In [ ]:
import sys
import shutil
from pathlib import Path

sys.path.append(str(Path().resolve().parent.parent))
from datamodule.dataset import ImageCSVDataset
from datamodule.splitter import KFoldSplit



dataset_path = Path().resolve().parent.parent / "data" / "MGS_data"
dataset = ImageCSVDataset(dataset_path)  # Replace with your actual dataset
splitter = KFoldSplit(k=3, seed=42)

train_indices, val_indices = splitter.get_splits(dataset)[0]  # Get the first split

lrp_sign_all = Path().resolve().parent.parent / "explainability" / "results" / "nbg" / "sign_all"
lrp_sign_pos = Path().resolve().parent.parent / "explainability" / "results" / "nbg" / "sign_positive"
lrp_val_sign_all = Path().resolve().parent.parent / "explainability" / "results" / "nbg" / "val" / "sign_all"
lrp_val_sign_pos = Path().resolve().parent.parent / "explainability" / "results" / "nbg" / "val" / "sign_positive"

lrp_val_sign_all.mkdir(parents=True, exist_ok=True)
lrp_val_sign_pos.mkdir(parents=True, exist_ok=True)
print(f"train_indices: {len(train_indices)}, val_indices: {len(val_indices)}")
for idx in val_indices:
    _, image_name, _ = dataset[idx]
    image_stem = image_name.stem
    
    # print(f"Processing {image_stem} for LRP results...")
    pattern = f"{image_stem}_lrp*"
    src_all_matches = list(lrp_sign_all.glob(pattern))
    src_all = src_all_matches[0] if src_all_matches else None
    src_pos_matches = list(lrp_sign_pos.glob(pattern))
    scr_pos = src_pos_matches[0] if src_pos_matches else None
        
    if not src_all.exists() and not scr_pos.exists():
        print(f"[WARN] No LRP files found for {image_name.name}")
        continue
    
    dst_src_all = lrp_val_sign_all / src_all.name
    dst_src_pos = lrp_val_sign_pos / scr_pos.name
    
    if src_all.exists():
        shutil.copy(src_all, dst_src_all)
    
    if scr_pos.exists():
        shutil.copy(scr_pos, dst_src_pos)